In [72]:
!pip install simpy
import simpy
import random
import math
import numpy as np

In [73]:
Total_Time = 100

RANDOM_SEED = 1
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300
    
total_sick = 0
    
num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

print(num_of_individuals, exponential_rate, num_of_beds)

1531 5.1033333333333335 128


Define the necessary set of arrays for bookkeeping

In [74]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
daily_patient = []
daily_fullness = []

In [75]:
class Patient(object):
    def __init__(self, name, env, opr, isInitial):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.hospital2home_treating_time_rate = 6*self.r
        self.isInitial = isInitial
        self.action = env.process(self.try_to_heal())
    
    def try_to_heal(self):
        if(not self.isInitial and self.is_going_hospital() == False):
            print('%s\thome    \t%g\t%g' % (self.name, math.ceil(self.env.now),bed.count))
            yield self.env.process(self.healing())
            print('%s\thealed  \t%g\t%g' % (self.name, math.ceil(self.env.now),bed.count))
        else:
            with bed.request() as req:
                yield req
                if not self.isInitial:
                  print('%s\thospital\t%g\t%g' % (self.name, math.ceil(self.env.now),bed.count))
                yield self.env.process(self.healing())
                print('%s\thealed  \t%g\t%g' % (self.name, math.ceil(self.env.now),bed.count))
            
    def healing(self):
        if((self.is_going_hospital and bed.count != num_of_beds) or self.isInitial):
            duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            print("%s\treject  %g\t%g" %(self.name, math.ceil(self.env.now),bed.count))
            duration = random.expovariate(1.0/self.hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(math.ceil(duration))
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [76]:
def patient_generator(env, bed, total_sick):
    """Generate new patients with using exponential rate!!!."""
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [77]:
def patient_generator_half_full(env, bed, total_sick):
    for i in range (num_of_beds // 2):
        yield env.timeout(0)
        patient = Patient('Initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [78]:
def patient_generator_full(env, bed, total_sick):
    for i in range (num_of_beds):
        yield env.timeout(0)
        patient = Patient('Initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate)) 
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [79]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator_half_full(env, bed, total_sick))
env.run(until = Total_Time)

Patient 2	home    	0	65
Patient 3	home    	0	65
Patient 4	home    	0	65
Patient 5	home    	0	65
Patient 6	home    	0	65
Patient 1	hospital	0	66
Patient 7	hospital	0	66
Initial 62	healed  	1	66
Initial 10	healed  	1	65
Initial 56	healed  	1	64
Initial 13	healed  	1	63
Initial 28	healed  	1	62
Initial 36	healed  	1	61
Initial 50	healed  	1	60
Initial 4	healed  	1	59
Patient 5	healed  	1	58
Initial 21	healed  	1	58
Initial 30	healed  	2	57
Initial 57	healed  	2	56
Patient 4	healed  	2	55
Initial 64	healed  	2	55
Initial 12	healed  	2	54
Initial 16	healed  	2	53
Initial 8	healed  	2	52
Initial 15	healed  	2	51
Initial 60	healed  	2	50
Initial 44	healed  	2	49
Initial 17	healed  	3	48
Initial 25	healed  	3	47
Initial 53	healed  	3	46
Initial 32	healed  	3	45
Patient 2	healed  	3	44
Patient 8	hospital	3	45
Initial 38	healed  	4	45
Patient 3	healed  	4	44
Initial 14	healed  	4	44
Initial 7	healed  	4	43
Patient 9	home    	4	42
Patient 10	home    	4	42
Patient 11	home    	4	42
Patient 12	home 

In [80]:
print(service_times)
print(str(np.mean(service_times)))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 3, 7, 3, 3, 7, 2, 2, 7, 1, 2, 8, 3, 3, 1, 3, 8, 2, 9, 9, 9, 2, 2, 4, 10, 10, 10, 10, 1, 11, 8, 1, 1, 1, 11, 2, 4, 11, 11, 2, 2, 2, 7, 12, 12, 7, 5, 3, 3, 2, 3, 3, 3, 13, 14, 7, 2, 14, 14, 5, 10, 5, 1, 4, 1, 5, 5, 2, 6, 1, 6, 12, 11, 2, 7, 2, 3, 10, 13, 17, 3, 17, 11, 1, 13, 8, 5, 2, 19, 5, 3, 9, 5, 10, 4, 7, 14, 10, 6, 8, 17, 8, 12, 1, 12, 12, 7, 6, 16, 9, 8, 14, 5, 25, 21, 5, 10, 6, 15, 4, 5, 3, 15, 9, 12, 5, 4, 13, 3, 13, 12, 1, 15, 9, 2, 7, 15, 1, 30, 3, 9, 5, 2, 2, 26, 3, 27, 10, 9, 3, 12, 5, 4, 4, 6, 1, 3, 3, 11, 9, 9, 8, 7, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 2, 1, 1, 2, 1, 1, 9, 3, 10, 9, 3, 4, 24, 4, 13, 3, 4, 3, 1, 5, 1, 5, 7, 4, 5, 3, 7, 22, 1, 3, 3, 3, 14, 22, 1, 9, 15, 1, 11, 2, 8, 1, 8, 3, 6, 2, 3, 11, 12, 3, 3, 14, 7, 4, 10, 1, 4, 5, 3, 3, 4, 6, 1, 3, 4, 2, 7, 15, 27, 1, 7, 13, 1, 11, 2, 2, 8, 2, 22, 2, 2, 8, 4, 3, 5, 3, 3, 1, 7, 3, 4, 1,

In [81]:
print(daily_patient)
print(str(np.mean(daily_patient)))

[7, 0, 0, 1, 6, 14, 1, 12, 0, 1, 23, 8, 1, 0, 17, 7, 2, 6, 0, 1, 5, 4, 4, 5, 1, 0, 3, 5, 4, 15, 2, 1, 1, 1, 1, 17, 6, 11, 2, 1, 8, 2, 2, 18, 7, 11, 2, 3, 7, 28, 2, 4, 6, 1, 1, 2, 10, 2, 9, 0, 3, 21, 1, 2, 5, 4, 5, 4, 8, 0, 7, 2, 9, 5, 9, 4, 12, 1, 28, 0, 1, 3, 4, 5, 2, 2, 23, 10, 9, 10, 0, 5, 7, 2, 2, 4, 1, 0, 2, 2]
5.45


In [82]:
print(daily_fullness)
print(str(np.mean(daily_fullness)))
print(daily_fullness.count(0))

[63, 57, 48, 44, 42, 32, 31, 28, 27, 23, 20, 16, 14, 12, 9, 8, 9, 8, 10, 8, 8, 7, 7, 7, 7, 5, 3, 2, 2, 3, 3, 3, 3, 2, 2, 1, 3, 4, 5, 4, 3, 4, 3, 2, 9, 9, 10, 8, 7, 8, 9, 10, 10, 9, 6, 5, 4, 4, 4, 3, 3, 5, 5, 3, 1, 0, 0, 1, 1, 3, 2, 2, 2, 4, 3, 3, 3, 3, 1, 9, 7, 6, 4, 4, 4, 2, 2, 3, 3, 4, 3, 3, 3, 5, 3, 1, 0, 1, 1, 1]
8.31
3
